<img src="https://global.utsa.edu/tec-partnership/images/logos/logotipo-horizontal-azul-transparente.png"  width="600">

## **Avance 4: Proyecto Integrador**
## Visualización interactiva de calidad de aire en AR en aplicaciones móviles con análisis y forecasting con AI y ML
### **TC5035 - Proyecto Integrador (Gpo 10)**
### **Equipo #56**
#### Tecnológico de Monterrey
---
*   NOMBRE: Paulina Escalante Campbell
*   MATRÍCULA: A01191962


### **Objetivo**
---
**Optimización de hiperparámetros**

Cuando se trabaja en un proyecto de ML, es común explorar y comparar varios modelos para determinar cuál se ajusta mejor a los datos y produce las predicciones más precisas. Una vez que has establecido un modelo de referencia o baseline, es momento de construir modelos alternativos. Estos modelos pueden diferir en términos de complejidad, enfoque matemático, suposiciones subyacentes y la manera en que manejan los datos.

> Una vez que has obtenido un número considerable de modelos alternativos, enfócate en los más prometedores en términos de la métrica de desempeño que es importante para tu caso de negocio. Ahora necesitas ajustarlos…

Este avance implica construir múltiples modelos (individuales, no ensambles) relevantes para resolver el problema y evaluar su desempeño. Diferentes algoritmos pueden comportarse de manera óptima en diferentes tipos de datos o tareas. La construcción de modelos alternativos permite explorar y evaluar cuál de ellos proporciona el mejor rendimiento para un problema particular.
Además, los modelos se pueden ajustar para determinar si se puede mejorar su rendimiento. Diferentes configuraciones de hiperparámetros pueden afectar significativamente el rendimiento de un modelo. Construir modelos alternativos implica explorar y ajustar estos hiperparámetros para encontrar la configuración óptima.

Las siguientes son acciones que deberás abordar en este avance:
* Construir al menos 6 modelos diferentes (individuales, no ensambles), utilizando algoritmos variados.
* Comparar el rendimiento de los modelos obtenidos.
* Seleccionar los dos modelos que proporcionen el mejor rendimiento.
* Ajustar los dos mejores modelos.
* Elegir el modelo individual final.



### **Dataset Inicial**
---
Global Air Quality Dataset 🌍
Comprehensive Air Quality Measurements from Major Cities Worldwide 🌍
https://www.kaggle.com/datasets/sazidthe1/global-air-pollution-data/data

### Diccionario de variables del dataset de calidad del aire

| Columna               | Descripción                                                                                 |
|-----------------------|---------------------------------------------------------------------------------------------|
| `country_name`        | Name of the Country                                                                         |
| `city_name`           | Name of the City                                                                            |
| `aqi_value`           | Overall AQI value of the city                                                               |
| `aqi_category`        | Overall AQI category of the city                                                            |
| `co_aqi_value`        | AQI value of Carbon Monoxide of the city                                                    |
| `co_aqi_category`     | AQI category of Carbon Monoxide of the city                                                 |
| `ozone_aqi_value`     | AQI value of Ozone of the city                                                              |
| `ozone_aqi_category`  | AQI category of Ozone of the city                                                           |
| `no2_aqi_value`       | AQI value of Nitrogen Dioxide of the city                                                  |
| `no2_aqi_category`    | AQI category of Nitrogen Dioxide of the city                                               |
| `pm2.5_aqi_value`     | AQI value of Particulate Matter (≤ 2.5 micrometers) of the city                             |
| `pm2.5_aqi_category`  | AQI category of Particulate Matter (≤ 2.5 micrometers) of the city                          |



### **Leer archivos, imports y google cloud drive**

In [ ]:
# Setup inicial del proyecto con GPU y google drive, conectar a runtime de T4GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

import psutil

ram_gb = psutil.virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

from google.colab import drive
drive.mount('/content/drive')

# Asegurarse que los datos han sido copiados a este directorio de google drive
import os
DIR = "/content/drive/MyDrive/Colab Notebooks/ProyectoIntegrador"
os.chdir(DIR)

Tue Oct 14 03:17:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Imports para análisis de datos y visualizaciones
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px

# Networking imports
import requests
import time
import unicodedata

# Normalizing
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer
from sklearn.feature_selection import VarianceThreshold, SelectKBest, chi2, f_classif, mutual_info_classif
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, learning_curve

# Models
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
import pickle

# characteristics
from sklearn.feature_selection import (
    SelectKBest, f_regression, mutual_info_regression,
    RFE, SequentialFeatureSelector
)
from sklearn.inspection import permutation_importance
from sklearn.decomposition import PCA
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score
)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Usar el dataset final del Avance 2 (con features seleccionadas), datos ya están preprocesados con el Avance 1, 2 y 3
df_selected = pd.read_csv('data_features_selected_clean.csv')  # 7 features seleccionadas + target

# Exploramos el modelo con dos posibles features
df = df_selected
df

,co_aqi_value,ozone_aqi_value,distance_from_equator,pm25_no2_ratio,ozone_co_ratio,country_mean_aqi,aqi_value
0,-0.257657,-1.010310,-1.609334,-0.429844,-1.493037,-0.836505,-0.609049
1,-0.257657,0.014283,0.441423,-0.399228,0.213691,-0.452705,-0.247782
2,-0.257657,-0.136393,1.461118,-0.440050,-0.037298,-0.663941,-0.710203
3,-0.257657,-0.106258,1.220933,-0.531899,0.012900,-0.750846,-0.305584
4,-0.257657,-0.287068,1.336271,-0.684980,-0.288288,-0.828714,-0.276683
...,...,...,...,...,...,...,...
16754,0.898454,3.660628,0.148227,-0.042038,2.271804,0.994122,1.110582
16755,-0.257657,0.014283,1.206799,-0.123682,0.213691,-0.328210,-0.175528
16756,0.898454,3.479818,-0.205231,0.403599,2.121211,1.673678,1.457398
16757,-0.257657,-0.558284,1.188464,-0.678176,-0.740068,-0.661446,-0.478993


In [ ]:
df_selected.head(1)

,co_aqi_value,ozone_aqi_value,distance_from_equator,pm25_no2_ratio,ozone_co_ratio,country_mean_aqi,aqi_value
0,-0.257657,-1.01031,-1.609334,-0.429844,-1.493037,-0.836505,-0.609049


In [ ]:
df.shape

(16759, 7)

#**Sección 1: Comparativa de Modelos**
---

La variable objetivo para el modelo es **aqi_value**, en el avance 3 se exploraron diferentes modelos para usar como baseline ya que no estabamos seguros de los datos y su comportamiento. Esta sección explorará algunos de esos modelos y unos nuevos.

Modelos propuestos:

* Linear Regression (baseline simple)
* Ridge Regression (regularización L2)
* Lasso Regression (regularización L1 + selección de features)
* Decision Tree Regressor (no lineal, interpretable)
* Random Forest Regressor (ensemble de árboles, robusto)
* Gradient Boosting Regressor (XGBoost o LightGBM - state-of-the-art)
* Support Vector Regressor (SVR) (efectivo en espacios de alta dimensionalidad)
* K-Nearest Neighbors (KNN) (basado en instancias)

Métricas a comparar:

* Principal: RMSE (Root Mean Squared Error) - penaliza errores grandes
* Secundaria 1: MAE (Mean Absolute Error) - interpretable en unidades de AQI
* Secundaria 2: R² Score - % de varianza explicada
* Tiempo de entrenamiento (segundos)


In [ ]:
# Test

### **Linear Regression (baseline simple)**

In [ ]:
# Test

### **Ridge Regression (regularización L2)**

In [ ]:
# Test

### **Lasso Regression (regularización L1 + selección de features)**

In [ ]:
# Test

### **Decision Tree Regressor (no lineal, interpretable)**

In [ ]:
# Test

### **Random Forest Regressor (ensemble de árboles, robusto)**

In [ ]:
# Test

### **Gradient Boosting Regressor (XGBoost o LightGBM)**

In [ ]:
# Test

### **Support Vector Regressor (SVR) (efectivo en espacios de alta dimensionalidad)**

In [ ]:
# Test

### **K-Nearest Neighbors (KNN) (basado en instancias)**

In [ ]:
# Test

#**Sección 2: Ajuste Fino de Hiperparámetros**
---

Para los 2 mejores modelos, usaremos:

* GridSearchCV o RandomizedSearchCV con validación cruzada
* Ajustar hiperparámetros clave según el algoritmo:
  * Random Forest: n_estimators, max_depth, min_samples_split
  * XGBoost: learning_rate, max_depth, n_estimators, subsample
  * SVR: C, epsilon, kernel

In [ ]:
# Test

#**Sección 3: Selección del Modelo Final**
---
Criterios de decisión:

* Performance en métricas principales
* Trade-off precisión vs tiempo de inferencia
* Interpretabilidad (importante para stakeholders)
* Robustez ante overfitting
* Aplicabilidad en producción


In [ ]:
#Test

###**Sección 3.1**

In [ ]:
# Test

# **Conclusiones**

---

Se estableció desempeño mínimo basado en:
* (1) baseline ingenuo (DummyRegressor, R²≈0)
